In [1]:
import os
import tensorflow as tf

from modules.run import load_config
from modules.run import Trainer, Metrics
from modules.data import DataManager
from modules.models import pretrained_cnn, pretrained_cnn_multichannel

## Testing imports

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [5]:
def run_experiment_from_config(config_file, country):

    config = load_config(config_file)
    data_manager = DataManager(config)

    class_weight = None
    train_generator = None
    val_generator = None

    if country == 'kenya':
        train_generator, val_generator, dataframe = data_manager.generate_kenya()
        class_weight = data_manager.class_weight("kenya")
    elif country == 'peru':
        train_generator, val_generator, dataframe = data_manager.generate_peru()
        class_weight = class_weight=[1.64, 1, 2]

    convnet = pretrained_cnn_multichannel(config, image_size=config["image_size"], n_channels=config["n_channels"])

    val_steps = config["sample"]["size"] * (config["validation_split"]) // config["batch_size"] + 1

    labels = None
    if config['mask'] is not None:
        epochs = 0
        labels = []
        for data, label in val_generator:
            if epochs >= val_steps:
                break
            labels.extend(np.argmax(label, axis=1))
            epochs += 1
        labels = np.array(labels)
    trainer = Trainer(config)
    metrics_callback = Metrics(val_generator, trainer.tensorboard_dir, labels, val_steps)
    trainer.callbacks.append(metrics_callback)

    convnet.compile(loss=trainer.loss, optimizer=trainer.optimizer, metrics=config["weighted_metrics"])

    convnet.fit_generator(
        train_generator, 
        config["sample"]["size"] * (1 - config["validation_split"]) // config["batch_size"] + 1,
        epochs=config["n_epochs"],
        callbacks=trainer.callbacks,
        validation_data=val_generator, 
        validation_steps=val_steps,
        class_weight=class_weight,
        use_multiprocessing=True
    )

In [6]:
run_experiment_from_config("cls_final_xception_kenya_rgb", "kenya")
run_experiment_from_config("cls_final_xception_peru_rgb", "peru")
run_experiment_from_config("cls_final_xception_kenya_masked", "kenya")
run_experiment_from_config("cls_final_xception_kenya_masked-inverted", "kenya")
run_experiment_from_config("cls_final_xception_kenya_two_with_mask", "kenya")

Declouded dataframe length: 152505
Found 15000 validated image filenames belonging to 3 classes.
Found 15000 validated image filenames belonging to 3 classes.
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
2

Process Keras_worker_ForkPoolWorker-4:
Traceback (most recent call last):
  File "/home/BenChoi/anaconda3/envs/tf2/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/BenChoi/anaconda3/envs/tf2/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/BenChoi/anaconda3/envs/tf2/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/BenChoi/anaconda3/envs/tf2/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/home/BenChoi/anaconda3/envs/tf2/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/BenChoi/anaconda3/envs/tf2/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/BenChoi/anaconda3/envs/tf2/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = 

KeyboardInterrupt: 